In [1]:

# 필요한 라이브러리

import numpy as np
from IPython import display
import sys
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import display, clear_output
import time
import random
import glob
from PIL import Image
from scipy.special import expit
import os


#출력 이쁘게 해주는 함수

def print_over(text):
    sys.stdout.write('\r' + text)
    sys.stdout.flush()
def generate_formatted_string(number):
    # 입력된 숫자를 정수와 소수 부분으로 분리합니다.
 
    number = f"{number:.14f}"

    int_part, decimal_part = str(number).split('.')
    decimal_part = decimal_part[:13]
    if len(int_part) > 3:
        format_string = 'OUT!OFVALUEPLEASE'
    # 정수 부분을 1000자리로 맞추기 위해 필요한 '0'의 개수를 계산합니다.
    else:
        num_zeros_needed = max(0, 3 - len(int_part))
        int_part = int_part[-3:]
        # 1000자리의 '0'과 정수 부분, 소수 부분을 합쳐서 문자열을 생성합니다.
        format_string = '0' * num_zeros_needed + int_part + '.' + decimal_part.ljust(13, '0')

    return format_string
    
# 필요한 함수 정의

# 활성 함수

#렐루
def relu(x):
    return np.maximum(0, x)
def relu_derivative(x):
    return np.where(np.array(x) > 0, 1, 0)

#리키렐루
def leaky_relu(x, alpha=0.001):
    return np.maximum(alpha * x, x)

def leaky_relu_derivative(x, alpha=0.001):
    return np.where(np.array(x) > 0, 1, alpha)

#시그모이드
def sigmoid(x):
    return 1000 / (1000 * (1 + np.exp(-x)))
def sigmoid_derivative(x):
    return 1 / (1 + np.exp(-x)) * (1 - 1 / (1 + np.exp(-x)))

#항등함수
def identity_function(x):
    return x

def identity_function_derivative(x):
    return np.ones(x.shape)

#소프트맥스 함수
def softmax(x):
    e_x = np.exp(x - np.max(x))  # overflow 방지
    return e_x / e_x.sum()

def softmax_derivative(x):
    softmax_x = softmax(x)
    return softmax_x * (1 - softmax_x)


#활성함수[0] = 원함수
#활성함수[1] = 도함수

sig = sigmoid, sigmoid_derivative
ralo = relu, relu_derivative
leaky_ralo = leaky_relu, leaky_relu_derivative
identity = identity_function, identity_function_derivative
soft = softmax, softmax_derivative


# 로스 함수

#MSE
def mean_squared_error(output, target):
    return np.mean((output - target) ** 2)
def mse_derivative(output, target):
    return 2 * (output - target)

#BSE
def cross_entropy_loss(y_pred, y_true):
    epsilon = 1e-15
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
    return -np.sum(y_true * np.log(y_pred))

mse = mean_squared_error, mse_derivative


# 신경망 초기화때 사용할 분포들

def gaussian_distribution(input1, input2, multiple, plus):
    return np.random.randn(input1, input2) * multiple + plus

def zero_distribution(input1, input2):
    return np.zeros((input1, input2))

def one_distribution(input1, input2, multiple):
    return np.ones((input1, input2)) * multiple

# 오차역전파에서 쓸 함수

def gradient_desent(learning_rate, gradient):        
    return learning_rate * gradient

def matrix_product_PI(function1, function2, f, K):
    if K == 0:
        return 1
    else:
        x = (function1[f - 1]).T * np.array(function2[f -1])
        for k in range(K-1):
            y = (function1[f - (k + 2)]).T * np.array(function2[f - (k+2)])
            x = x @ y
            
    return x

def grape_show(x_values, y_values_funk, y_values_square):
    plt.plot(x_values, y_values_funk, label='Funk Object')
    plt.plot(x_values, y_values_square, label='want_funtion', linestyle='--', color='red')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title('Funk Object vs. want_funtion')
    plt.legend()
    plt.grid(True)
    plt.ylim(-1.1, 1.1)  
    plt.show()

In [2]:
#Mnist 데이터셋에서 임의로 Parameter1 만큼의 갯수만 가져오기

def get_random_png_files(folder_path, num_files):
    png_files = glob.glob(os.path.join(folder_path, "*.png"))
    random_files = random.sample(png_files, num_files)
    return random_files

folder_path6 = "C:/Users/Persist/Downloads/mnist_png.tar/mnist_png/training/6"  
random_png_files6 = get_random_png_files(folder_path6, 5000)

folder_path3 = "C:/Users/Persist/Downloads/mnist_png.tar/mnist_png/training/3"  
random_png_files3 = get_random_png_files(folder_path3, 5000)

folder_path_testing_3 = "C:/Users/Persist/Downloads/mnist_png.tar/mnist_png/testing/3"  
random_png_files_test_3 = get_random_png_files(folder_path_testing_3, 500)


ranchooseimage6 = random.sample(random_png_files6, 1)
ranchooseimage3 = random.sample(random_png_files3, 1)
ranchooseimage_test_3 = random.sample(random_png_files_test_3, 1)

#print(ranchooseimage)

#이미지를 열고 행렬로 변환

def process_image(image_path):
    # 이미지 열기
    with Image.open(image_path) as image:
        # 이미지의 해상도 (너비, 높이) 저장
        width, height = image.size
        Weight = width
        Height = height
        
        # 이미지를 그레이스케일로 변환
        gray_image = image.convert("L")
        
        # 그레이스케일 이미지를 numpy 배열로 변환
        gray_array = np.array(gray_image).reshape(1, -1)
        
        return Height, Weight, gray_array
    
DataSet6 = []
DataSet3 = []
DataSet_test_3 = []

for i in range(5000):
    random_image_path6 = random_png_files6[i]
    _, _, gray_array = process_image(random_image_path6)
    normalized_array = (gray_array / 255.0)  # 255로 나누어 정규화
    DataSet6.append(normalized_array)
    
for i in range(5000):
    random_image_path3 = random_png_files3[i]
    _, _, gray_array = process_image(random_image_path3)
    normalized_array = (gray_array / 255.0)  # 255로 나누어 정규화
    DataSet3.append(normalized_array)
    
    
for i in range(500):
    random_image_path_test3 = random_png_files_test_3[i]
    _, _, gray_array = process_image(random_image_path_test3)
    normalized_array = (gray_array / 255.0)  # 255로 나누어 정규화
    DataSet_test_3.append(normalized_array)



In [3]:
#신경망
#신경망 = Neurel_Network() 로 선언
#신경망([p0,p1,p2,...,pk+1], weight 초깃값, bias 초깃값, 은닉층 활성함수, 출력 활성함수)


class Neurel_Network:
    def __init__(self):
        self.layer_number = 0
        self.weight = []
        self.bias = []
        self.layer = []
        self.after_layer = []
        self.external_active_function = []
        self.external_last_active_function = []
        self.final_layer = []
            
    def active_function(self, input, i): # i = 0이면 원래 함수, i = 1이면 미분된 함수
        function = self.external_active_function[i]
        return function(input)
    
    def last_active_function(self, input, i):
        function = self.external_last_active_function[i]
        return function(input)

    def initialize_network(self, perceptron, weight_initial, bias_initial, function, last_function):
        self.layer_number = len(perceptron)
        self.external_active_function = function
        self.external_last_active_function = last_function
        
        for i in range((self.layer_number) - 1):
            self.weight.append(weight_initial(perceptron[i], perceptron[i + 1], 0.1, 0))
            self.bias.append(bias_initial(1, perceptron[i + 1]))

    def forward(self, input):
        #after_layer[k] = Active_function(L[k])
        
        self.layer = []
        self.after_layer = []
        self.final_layer = []
        self.after_layer.append(np.array(input).reshape(1, -1))
        
        self.layer.append(np.array(input).reshape(1,-1)) #L[0] = input
        #L[1]~L[k]
        for i in range((self.layer_number) - 2):
            #i = 0 일떄, l[1] = layer[0] * w[0]
            self.layer.append(np.array((self.after_layer[i] @ self.weight[i]) + self.bias[i]))
            self.after_layer.append(np.array(self.active_function(self.layer[i+1], 0)))
            
        self.layer.append(np.array((self.layer[(self.layer_number) - 2] @ self.weight[(self.layer_number) - 2]) + self.bias[(self.layer_number) - 2]))
        self.after_layer.append(np.array(self.last_active_function(self.layer[(self.layer_number) - 1], 0)))
        output = self.after_layer[(self.layer_number)-1]
        
        #Final_layer
        for i in range(len(self.layer) - 1):
            self.final_layer.append(np.array(self.active_function(self.layer[i], 1)))
        
        self.final_layer.append(np.array(self.last_active_function(self.layer[len(self.layer) - 1], 1)))

        return output



In [4]:
#오차역전파
def backward(neural_network, Loss_function, inputdataset, targetdataset, learning_rate):

        
    minibatch = len(inputdataset)
        

    #초기화
    gradient_weight = [0]*((neural_network.layer_number)-1)
    gradient_bias= [0]*((neural_network.layer_number)-1)
    
    weight = neural_network.weight
    bias = neural_network.bias
    
    
    for iteration_minibatch in range(minibatch):
        
        
        input = inputdataset[iteration_minibatch]
        target_data = targetdataset[iteration_minibatch]
        #오차역전파 시작
        output = neural_network.forward(input)
        
        #weight[k], bias[k]를 갱신함
        gradient_weight[neural_network.layer_number-2] =\
        neural_network.after_layer[neural_network.layer_number-2].T\
        @Loss_function[1](output, target_data)\
        *neural_network.final_layer[neural_network.layer_number - 1]
        
        gradient_bias[neural_network.layer_number-2]=\
        Loss_function[1](output, target_data)\
        *neural_network.final_layer[neural_network.layer_number - 1]

        weight[neural_network.layer_number-2] -= learning_rate * gradient_weight[neural_network.layer_number-2]
        bias[neural_network.layer_number-2] -= learning_rate * gradient_bias[neural_network.layer_number-2]

        #이후부터 하나씩 갱신해감

        for i in range(1,(neural_network.layer_number)-1):
            #output = neural_network.forward(input)
                
            gradient_weight[neural_network.layer_number - 2-i]=\
            neural_network.after_layer[neural_network.layer_number-2 -i].T\
            @(Loss_function[1](output, target_data)\
            *neural_network.final_layer[neural_network.layer_number - 1])\
            @matrix_product_PI(neural_network.weight, neural_network.final_layer, neural_network.layer_number-1, i)
            
            gradient_bias[neural_network.layer_number - 2-i]=\
            Loss_function[1](output, target_data)\
            *neural_network.final_layer[neural_network.layer_number - 1]\
            @matrix_product_PI(neural_network.weight, neural_network.final_layer, neural_network.layer_number-1, i)
            
            weight[neural_network.layer_number - 2-i] -= learning_rate * gradient_weight[neural_network.layer_number - 2-i] / minibatch
            bias[neural_network.layer_number - 2-i] -= learning_rate * gradient_bias[neural_network.layer_number - 2-i] / minibatch
        
    neural_network.weight = weight
    neural_network.bias = bias


                

    return Loss_function[0](input, target_data)

    


In [5]:
N = Neurel_Network()

N.initialize_network([784, 1024, 1024,512, 1], gaussian_distribution, zero_distribution, leaky_ralo, sig)

len_data = len(DataSet3)

loss_values = []
wantrange = 100000


In [6]:
#Data divide to MiniBatch
def minibatch_learning(dataset, batch_size, type):
    dataset_len = len(dataset)
    minibatchset = []
    if dataset_len % batch_size == 0:
        batch_number = dataset_len // batch_size
        last_iteration = 0
    else:
        batch_number = dataset_len // batch_size + 1
        last_iteration = dataset_len % batch_size
    
   # if type == 'suffle':
        
    if type == 'continue':
        if last_iteration == 0:
            for i in range(batch_number):
                A = []
                for k in range(batch_size):
                 A.append(dataset[i*batch_size + k])
                minibatchset.append(A)
    
    return minibatchset
    


    # if len(inputdataset) % batchsize == 0:
    #     batch_number = len(inputdataset) // batchsize
    #     last_iteration = 0
    #     minibatch = batchsize
    # else:
    #     batch_number = len(inputdataset) // batchsize + 1
    #     last_iteration = len(inputdataset) % batchsize

In [7]:

#def backward(neural_network, Loss_function, inputdataset, targetdataset, batchsize, learning_rate):

real_data = [1] * 5000
fake_data = [0] * 5000


DataSetSuffle = DataSet3 + DataSet6
CorrcetSetSuffle =  fake_data + real_data



Datacombined = list(zip(DataSetSuffle, CorrcetSetSuffle))
random.shuffle(Datacombined)
DataSetSuffle, CorrcetSetSuffle = zip(*Datacombined)

Inputdataset = minibatch_learning(DataSetSuffle, 10, 'continue')
CorrectDataset = minibatch_learning(CorrcetSetSuffle, 10, 'continue')

In [11]:

for i in range(wantrange):
    
    lr = 0.0001
    what = np.random.choice([0,1])
    k = i % 5000
    loss = backward(N, mse, Inputdataset[k], CorrectDataset[k], lr)
    loss_values.append(loss)
    output = N.forward(DataSet6[k])
    loss = generate_formatted_string(loss)
    print_over(f"Epoch: {i} / {wantrange} | Loss: {loss} | Output(6): {output}")        


        
    # else:          
    #     k = np.random.choice(len(DataSet3_mini) -1)


    #     # k = np.random.choice(np.arange(0, 5000))

    #     target = [0]*len_data
    #     loss = backward(N, mse, DataSet3_mini[k], target,  lr)
    #     loss_values.append(loss)
    #     output = N.forward(DataSet3[k])
    #     loss = generate_formatted_string(loss)
    #     #output = generate_formatted_string(output)
    #     print_over(f"Epoch: {i} / {wantrange} | Loss: {loss} | Output(3): {output}")



Epoch: 30 / 100000 | Loss: 000.1029225023342 | Output(6): [[2.45513949e-05]]

KeyboardInterrupt: 

In [ ]:

# for k in range(100):
#     print(N.forward(DataSet3[k]))

# print('-----------------------------------------------')
# for k in range(100):
    
#     print(N.forward(DataSet6[k]))
    

real = 0
fake = 0
non = 0

can = 10000
for i in range(can):

    k = np.random.choice(np.arange(1, 500))
    result = N.forward(DataSet_test_3[k])
    # result = N.forward(DataSet6[k])
    if result < 0.11:
        real += 1
    elif  result > 0.89:
        fake += 1
    else:
        non += 1

print(real, "/", can, non)

#대충 15초 걸림

In [ ]:

# 그래프 그리기
plt.plot(range(len(loss_values)), loss_values, label='Loss')
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.legend()
plt.title('Loss vs. Iteration')
plt.ylim(0, 10000)
plt.show()

# 그래프 그리기
plt.plot(range(len(loss_values)), loss_values, label='Loss')
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.legend()
plt.title('Loss vs. Iteration')
plt.ylim(0, 10)
plt.show()

# 그래프 그리기
plt.plot(range(len(loss_values)), loss_values, label='Loss')
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.legend()
plt.title('Loss vs. Iteration')
plt.ylim(0, 1)
plt.show()

# 그래프 그리기
plt.plot(range(len(loss_values)), loss_values, label='Loss')
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.legend()
plt.title('Loss vs. Iteration')
plt.ylim(0, 0.1)
plt.show()

# 그래프 그리기
plt.plot(range(len(loss_values)), loss_values, label='Loss')
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.legend()
plt.title('Loss vs. Iteration')
plt.ylim(0, 0.0001)
plt.show()